In [1]:
import pandas as pd
# Load the CSV files into DataFrames
nfo_scrips_df = pd.read_csv('/home/bitsbytes2algoz/Desktop/new/NFO_symbols.csv')
mcx_scrips_df = pd.read_csv('/home/bitsbytes2algoz/Desktop/new/NFO_symbols.csv')

async def get_lotsize():
    global atm_strike
     
    while True:
        await asyncio.sleep(5)
        
        if exchange.upper() == 'NSE':
            df = nfo_scrips_df
        elif exchange.upper() == 'MCX':
            df = mcx_scrips_df
        else:
            raise ValueError("Unsupported exchange. Please use 'NSE' or 'MCX'.")

        # Find the first matching symbol
        match = df[df['Symbol'] == symbol]
        
        if not match.empty:
            # Get LotSize
            lotsize = match.iloc[0]['LotSize']
            
            # Check feedJson if token is provided
            if Initial_token:
                if not feedJson[Initial_token]:
                    logger.info(f"FeedJson for token {Initial_token} is empty. LotSize: {lotsize}.")                    
                else:
                    # Retrieve the last recent price
                    last_price = int(feedJson[Initial_token][-1]['ltp'])
                    # Calculate ATM strike price
                    mod = int(last_price) % 100
                    atm_strike = last_price - mod if mod <= 50 else last_price + (100 - mod)                    
                    # Find trading symbols for ATM strike
                    filtered_df = df[
                        (df['Symbol'] == symbol) &
                        (df['StrikePrice'] == float(atm_strike))
                    ]
                    
                    if filtered_df.empty:
                        logger.info(f"Could not find trading symbols for ATM strike {atm_strike}")
                        #return lotsize, last_price, None, None
                        logger.info(f"wait for lot size to update {lotsize}.")

                    # Find the CE and PE trading symbols
                    ce_row = filtered_df[filtered_df['OptionType'] == 'CE'].sort_values('Expiry').iloc[0]
                    pe_row = filtered_df[filtered_df['OptionType'] == 'PE'].sort_values('Expiry').iloc[0]

                    ce_trading_symbol, pe_trading_symbol = ce_row['TradingSymbol'], pe_row['TradingSymbol']
                    ce_trading_token, pe_trading_token = ce_row['Token'], pe_row['Token']                    
                    
                      # Update the state
                    state.ce_trading_symbol = ce_trading_symbol
                    state.pe_trading_symbol = pe_trading_symbol
                    state.ce_trading_token = ce_trading_token
                    state.pe_trading_token = pe_trading_token

                    #logger.info(f"CE Symbol: {ce_trading_symbol}, PE Symbol: {pe_trading_symbol}")            
        else:
            logger.warning(f"{symbol} not found on {exchange}.")

async def resample_ticks():
    global resampled_data
    
    while True:
        if not feedJson:
            await asyncio.sleep(1)
            continue

        with feed_lock:
            temp_resampled_data = {}
            for token, ticks in feedJson.items():
                try:
                    if ticks:
                        # Create a DataFrame with the new ticks
                        df_new = pd.DataFrame(ticks)
                        df_new["tt"] = pd.to_datetime(df_new["tt"])
                        df_new.set_index("tt", inplace=True)

                        # Determine the current resample interval
                        current_resample_interval = df_new.index.max().floor(resample_frequency)

                        # Initialize or update the resampled DataFrame
                        if token not in resampled_data:
                            # Initialize the DataFrame with the first resample
                            df_resampled = df_new['ltp'].resample(resample_frequency).ohlc()
                            temp_resampled_data[token] = df_resampled
                            last_resample_time[token] = df_resampled.index.max()
                        else:
                            # Create a copy of the existing data
                            temp_df = resampled_data[token].copy()                            
                            # Resample the new ticks
                            df_resampled = df_new['ltp'].resample(resample_frequency).ohlc()

                            # Update the temporary DataFrame with new data
                            for idx, row in df_resampled.iterrows():
                                if idx in temp_df.index:
                                    temp_df.loc[idx, 'high'] = max(temp_df.loc[idx, 'high'], row['high'])
                                    temp_df.loc[idx, 'low'] = min(temp_df.loc[idx, 'low'], row['low'])
                                    temp_df.loc[idx, 'close'] = row['close']
                                else:
                                    temp_df.loc[idx] = row

                            # Update the last resampled time
                            last_resample_time[token] = df_resampled.index.max()                            
                            temp_resampled_data[token] = temp_df

                        if token in temp_resampled_data:
                            supertrend, su_direction = numba_indicators.supertrend_numba(
                                temp_resampled_data[token]['high'].values,
                                temp_resampled_data[token]['low'].values,
                                temp_resampled_data[token]['close'].values
                                
                            )
                            temp_resampled_data[token]['supertrend'] = supertrend
                            temp_resampled_data[token]['su_direction'] = su_direction

                        if token in temp_resampled_data:
                            jma, ce_direction = numba_indicators.jma_numba_direction(
                                temp_resampled_data[token]['close'].values                        #       
                            )                            
                            temp_resampled_data[token]['jma'] = jma
                            temp_resampled_data[token]['ce_direction'] = ce_direction                            

                        # Clean up the data
                        temp_resampled_data[token] = temp_resampled_data[token].dropna(subset=['open', 'high', 'low', 'close'])

                except Exception as e:
                    if isinstance(e, KeyError) and e.args[0] == 'tt':
                        print(f"Market likely closed for token {token}")
                    else:
                        print(f"Error resampling data for token {token}: {e}")

            # After processing all tokens, update the global resampled_data
            resampled_data = temp_resampled_data
            #print(resampled_data)

        await asyncio.sleep(0.1)

from collections import defaultdict
completed_candles_dfs = defaultdict(pd.DataFrame)
last_processed_candle = defaultdict(pd.Timestamp)

async def candle_end_finder():
    global resampled_data, resample_frequency
    logger.info("Starting candle_end_finder")

    while True:
        current_time = pd.Timestamp.now()
        time_bucket_start = current_time.floor(resample_frequency)
        time_bucket_end = time_bucket_start + pd.Timedelta(resample_frequency)
        wait_time = (time_bucket_end - current_time).total_seconds() + 0.001

        if wait_time > 0:
            await asyncio.sleep(wait_time)

        data_copy = {}
        with feed_lock:
            if not resampled_data:
                logger.warning("resampled_data is empty.")
                continue

            # Copying the data outside of the lock to avoid data modification during processing
            for token, df in resampled_data.items():
                if df is None or df.empty:
                    logger.warning(f"DataFrame for token {token} is empty or None.")
                    continue

                # if len(df) < 3:
                #     logger.warning(f"Not enough data for token {token}. Need at least 3 candles.")
                #     continue

                data_copy[token] = df.copy()

        # Process each token's data
        for token, df in data_copy.items():
            logger.debug(f"Processing token: {token}")

            # Checking if this is the first time processing or if the bucket has moved forward
            if token not in last_processed_candle or last_processed_candle[token] < time_bucket_end:
                try:
                    previous_length = len(completed_candles_dfs[token]) if token in completed_candles_dfs else 0

                    if time_bucket_start == df.index[-1]:
                        # Only include the last candle if it matches the bucket start
                        completed_candles_dfs[token] = df
                        last_processed_candle[token] = time_bucket_start

                    elif time_bucket_end == df.index[-1]:
                        # If the end bucket matches, take all up to that point
                        completed_candles_dfs[token] = df.loc[:time_bucket_end - pd.Timedelta(microseconds=1)]
                        last_processed_candle[token] = time_bucket_start
                    else:
                        logger.error(f"No new candle forming now for token {token}")
                        continue
                                        
                    new_length = len(completed_candles_dfs[token])
                    rows_added = new_length - previous_length
                    
                    if rows_added > 1:
                        logger.warning(f"Multiple rows ({rows_added}) added for token {token} at {pd.Timestamp.now()}. "
                        f"Previous length: {previous_length}, New length: {new_length}")
                    # Verify that the required columns exist
                    try:
                        required_columns = ['ce_direction']
                        missing_columns = [col for col in required_columns if col not in completed_candles_dfs[token].columns]
                        if missing_columns:
                            raise KeyError(f"Missing columns {missing_columns}")

                    except KeyError as e:
                        logger.error(f"Error processing token {token}: {e}")

                except KeyError as e:
                    logger.error(f"Error processing token {token}: Column {e} not found.")
                except IndexError:
                    logger.error(f"Error processing token {token}: Not enough candles to calculate indicators.")

        # Calculate the time for the next bucket
        next_bucket_start = time_bucket_end
        wait_time = (next_bucket_start - pd.Timestamp.now()).total_seconds()
        if wait_time > 0:
            await asyncio.sleep(wait_time)

def get_latest_price_option(entry_instrument):
    entry_instrument_str = str(entry_instrument)
    with feed_lock:
        if entry_instrument_str in extra_feedJson:
            latest_data = extra_feedJson[entry_instrument_str][-1]
            latest_price = latest_data['ltp']
            
            return latest_price
        else:
            print(f"{entry_instrument_str} not found in extra_feedJson")  # Debug print
            return None
        
def get_latest_price_index(entry_instrument):
    entry_instrument_str = str(entry_instrument)
    with feed_lock:
        if entry_instrument_str in feedJson:
            latest_data = feedJson[entry_instrument_str][-1]
            latest_price = latest_data['ltp']
            
            return latest_price
        else:
            print(f"{entry_instrument_str} not found in feedJson")  # Debug print
            return None
        
import asyncio
import nest_asyncio
import logging
import pandas as pd
import pyotp 
from asyncio import CancelledError
from collections import defaultdict, deque
from threading import Thread
from threading import Lock
from datetime import datetime
import numba_indicators
import uuid
from NorenRestApiPy.NorenApi import NorenApi

# Initialize logger
logger = logging.getLogger('ShoonyaApi')
logger.setLevel(logging.INFO) 
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

file_handler = logging.FileHandler('shoonya_api.log') # Replace 'shoonya_api.log' with your desired log file name
file_handler.setLevel(logging.INFO)  # Set the logging level for the file handler
# Apply the same formatter to the file handler
file_handler.setFormatter(formatter)
# Add the file handler to the logger
logger.addHandler(file_handler) 

# Global variables
resampled_data = {}
last_resample_time = {}
monitoring_tasks = {}
last_processed_candle = {} 
current_positions = {}
feed_opened = False
feed_lock = Lock()
position_lock = asyncio.Lock()
feedJson = defaultdict(lambda: deque(maxlen=20))
extra_feedJson = defaultdict(lambda: deque(maxlen=20))
api = None
exchange = 'NSE'
Initial_token = '26009'
subscription_string = f"{exchange}|{Initial_token}"
resample_frequency = "15s"
symbol = "BANKNIFTY"
atm_strike = int

# TRADE CONTROL flags 
enable_call_trades = True
enable_put_trades = True
enable_all_trades = True
force_exit_triggered = False

class TradingState:
    def __init__(self):
        self.ce_trading_symbol = None
        self.pe_trading_symbol = None
        self.ce_trading_token = None
        self.pe_trading_token = None
state = TradingState()


def initialize_api(credentials_file="usercred.xlsx"):
    global api
    api = NorenApi(
        host="https://api.shoonya.com/NorenWClientTP/",
        websocket="wss://api.shoonya.com/NorenWSTP/"
    )

    credentials = pd.read_excel(credentials_file)
    user = credentials.iloc[0, 0]
    password = credentials.iloc[0, 1]
    vendor_code = credentials.iloc[0, 2]
    app_key = credentials.iloc[0, 3]
    imei = credentials.iloc[0, 4]
    qr_code = credentials.iloc[0, 5]
    factor2 = pyotp.TOTP(qr_code).now()

    api.login_result = api.login(
        userid=user,
        password=password,
        twoFA=factor2,
        vendor_code=vendor_code,
        api_secret=app_key,
        imei=imei
    )

def event_handler_order_update(data):
    logger.info(f"Order update: {data}")

def event_handler_feed_update(tick_data):
    #print(tick_data)
    try:
        if 'lp' in tick_data and 'tk' in tick_data:
            timest = datetime.fromtimestamp(int(tick_data['ft'])).isoformat()
            token = tick_data['tk']

            with feed_lock:  # Acquire lock for thread-safety

                    if token == Initial_token:
                        feedJson[token].append({'ltp': float(tick_data['lp']), 'tt': timest})
                    else:
                        extra_feedJson[token].append({'ltp': float(tick_data['lp']), 'tt': timest})

    except (KeyError, ValueError) as e:
        logger.error(f"Error processing tick data: {e}")


async def connect_and_subscribe():
    global feed_opened
    retry_delay = 1  # Initial retry delay in seconds
    max_retry_delay = 32  # Maximum retry delay in seconds
    while True:
        try:
            api.start_websocket(
                order_update_callback=event_handler_order_update,
                subscribe_callback=event_handler_feed_update,
                socket_open_callback=open_callback,
                socket_close_callback=close_callback
            )
            await wait_for_feed_open(timeout=30)  # Wait for feed to open with a timeout
            api.subscribe([subscription_string])
            logger.info("WebSocket connected and subscribed successfully.")
            retry_delay = 1  # Reset retry delay after successful connection
            await monitor_connection()
        except Exception as e:
            logger.error(f"WebSocket connection error: {e}")
            logger.info(f"Reconnecting in {retry_delay} seconds...")
            await asyncio.sleep(retry_delay)
            retry_delay = min(retry_delay * 2, max_retry_delay)  # Exponential backoff

async def wait_for_feed_open(timeout):
    global feed_opened
    start_time = asyncio.get_event_loop().time()
    while not feed_opened:
        if asyncio.get_event_loop().time() - start_time > timeout:
            raise TimeoutError("Timed out waiting for feed to open")
        await asyncio.sleep(1)

async def monitor_connection():
    global feed_opened
    while True:
        if not feed_opened:
            logger.warning("Feed closed unexpectedly. Reconnecting...")
            raise Exception("Feed closed")
        await asyncio.sleep(5)  # Check connection status every 5 seconds

def close_callback():
    global feed_opened
    feed_opened = False
    logger.warning("WebSocket connection closed.")

def open_callback():
    global feed_opened
    if not feed_opened:
        feed_opened = True
        logger.info('Feed Opened')
    else:
        logger.warning('Feed Opened callback called multiple times.')

async def main():
    initialize_api()
    websocket_task = asyncio.create_task(connect_and_subscribe())
    resample_task = asyncio.create_task(resample_ticks())
    lot_size_task = asyncio.create_task(get_lotsize()) 
    candle_end_finder_task = asyncio.create_task(candle_end_finder())     
    direction_change_event_handler_task = asyncio.create_task(direction_change_event_handler())
    process_direction_changes_task = asyncio.create_task(process_direction_changes())
    await asyncio.gather(websocket_task, lot_size_task, resample_task, candle_end_finder_task, direction_change_event_handler_task, process_direction_changes_task)
# Your existing event loop setup
loop = asyncio.get_event_loop()
if loop.is_running():
    nest_asyncio.apply()
asyncio.create_task(main())
if not loop.is_running():
    loop.run_forever()


direction_change_queue = asyncio.Queue()
async def direction_change_event_handler():
    last_changes = {}
    while True:
        for token, df in completed_candles_dfs.items():
            if df.empty:
                continue

            current_direction = df['ce_direction'].iloc[-1]
            second_direction = df['su_direction'].iloc[-1]   ########################################################################################
            last_direction = last_changes.get(token, {}).get('ce_direction', None)

            # Check if the 'ce_direction' value has changed
            if last_direction is None or current_direction != last_direction:
                logger.info(f"Direction change detected for token {token} at {df.index[-1]}: {current_direction}")
                await direction_change_queue.put((token, current_direction, last_direction, second_direction)) ########################################################################################

            # Update last changes
            last_changes[token] = {'index': df.index[-1], 'ce_direction': current_direction}

        await asyncio.sleep(0.001)


async def process_direction_changes():
    while True:
        token, current_direction, previous_direction, second_direction = await direction_change_queue.get() ########################################################################################
        try:
            await execute_trade_logic(token, current_direction, previous_direction, second_direction) ########################################################################################
        except Exception as e:
            logger.error(f"Error executing trade logic for {token}: {e}")

        direction_change_queue.task_done()

############################################################     
class TradeTracker:
    def __init__(self):
        self.total_trades = 0
        self.total_points_collected = 0
        self.total_points_lost = 0
        self.overall_profit_loss = 0
        self.win_trades = 0
        self.loss_trades = 0
        self.zero_pnl_trades = 0  # New attribute to track zero P/L trades

    def update_stats(self, profit_loss):
        self.total_trades += 1

        if profit_loss > 0:
            self.win_trades += 1
            self.total_points_collected += profit_loss
        elif profit_loss < 0:
            self.loss_trades += 1
            self.total_points_lost += abs(profit_loss)
        else:  # Handle zero P/L trades
            self.zero_pnl_trades += 1

        self.overall_profit_loss += profit_loss

    def print_summary(self):
        logger.info("Trade Summary:")
        logger.info(f"  Total Trades: {self.total_trades}")
        logger.info(f"  Win Trades: {self.win_trades}")
        logger.info(f"  Loss Trades: {self.loss_trades}")
        logger.info(f"  Zero P/L Trades: {self.zero_pnl_trades}")  # Add this line
        logger.info(f"  Total Points Collected: {self.total_points_collected:.2f}")
        logger.info(f"  Total Points Lost: {self.total_points_lost:.2f}")
        logger.info(f"  Overall Profit/Loss: {self.overall_profit_loss:.2f} points")

# Create a TradeTracker instance
trade_tracker = TradeTracker()

class Trade:
    def __init__(self, token, position_type, entry_price, option_token, symbol_name):
        self.token = token
        self.position_type = position_type
        self.entry_price = entry_price
        self.option_token = option_token
        self.symbol_name = symbol_name
        self.monitoring_task = None
        self.exit_event = asyncio.Event()
        self.exited = False  # Flag to indicate if the trade has exited

        # Initialize your strategy parameters here (target, stop loss, etc.)
        self.highest_price = entry_price
        self.target_price = entry_price + 20
        self.sl_price = entry_price - 7
        self.trailing_sl_price = self.sl_price
        self.trail_activated = False
        self.trail_activation_point = 4

    def update_sl(self, new_sl_price):
        """Update the stop loss price."""
        if not self.exited:
            self.sl_price = new_sl_price
            logger.info(f"SL updated for {self.symbol_name} to {self.sl_price}")

    def update_trailing_sl(self, new_trailing_sl_price):
        """Update the trailing stop loss price."""
        if not self.exited:
            self.trailing_sl_price = new_trailing_sl_price
            self.trail_activated = True ######################
            logger.info(f"Trailing SL updated for {self.symbol_name} to {self.trailing_sl_price}")

    async def force_exit(self, exit_reason="Forced Exit"):
        """Force exit the trade immediately."""
        if not self.exited:
            await exit_trade(self, exit_reason)
            logger.info(f"Trade forcibly exited for {self.symbol_name}")
        

async def monitor_position(trade):
    try:
        while not trade.exited:
            # Fetch real-time price 
            current_price = get_latest_price_option(trade.option_token)

            if current_price is None:
                await asyncio.sleep(0.05)
                continue

            # Check for target/stop loss/trailing stop loss (using trade's parameters)
            if current_price >= trade.target_price:
                await exit_trade(trade, "Target Reached")
                break  # Exit the loop after exiting the trade
            elif current_price <= trade.sl_price:
                await exit_trade(trade, "Stop Loss Hit")
                break  # Exit the loop after exiting the trade
            elif not trade.trail_activated and current_price >= (trade.entry_price + trade.trail_activation_point):
                trade.trail_activated = True
                trade.trailing_sl_price = trade.entry_price
                logger.info(f"Trailing SL activated for {trade.symbol_name} at {trade.trailing_sl_price}")
            elif trade.trail_activated and current_price <= trade.trailing_sl_price:
                await exit_trade(trade, "Trailing Stop Loss Hit")
                break

            # # Update highest_price for trailing stop loss
            if current_price > trade.highest_price:
                trade.highest_price = current_price
            #     if trade.trail_activated:
            #         trade.trailing_sl_price = trade.highest_price - (trade.trail_activation_point - 3)
##########################################################################################################################################################
            await asyncio.sleep(0.05)
    except asyncio.CancelledError:
        logger.info(f"Monitoring cancelled for {trade.token} due to normal exit")


async def enter_trade(token, position_type, entry_type):
    try:
        option_token = state.ce_trading_token if position_type == "call_buy" else state.pe_trading_token
        symbol_name = state.ce_trading_symbol if position_type == "call_buy" else state.pe_trading_symbol

        entry_price = get_latest_price_option(option_token)  # Assuming this function exists
        if entry_price is None:
            logger.error(f"Unable to get entry price for {symbol_name}")
            return

        # Create a new Trade object
        trade = Trade(token, position_type, entry_price, option_token, symbol_name)

        # Update current_positions
        current_positions[token] = trade

        action_time = pd.Timestamp.now()
        logger.warning(f"Trade entered at {action_time} - {symbol_name} at {entry_price}")

        # Create a unique identifier for the trade and its monitoring task
        trade_id = uuid.uuid4()

        # Start the new monitoring task immediately
        trade.monitoring_task = asyncio.create_task(monitor_position(trade))
        monitoring_tasks[trade_id] = trade.monitoring_task

        # Place actual order using your brokerage API (replace with your actual order placement logic)
        # order_result = place_order(symbol_name, position_type, lot_size, entry_price)  # Example
        # logger.info(f"Order placed: {order_result}")

    except Exception as e:
        logger.error(f"Error in enter_trade for {token}: {e}")
        raise

async def exit_trade(trade, exit_type):    
    try:
        exit_price = get_latest_price_option(trade.option_token)  # Assuming this function exists
        if exit_price is None:
            logger.error(f"Unable to get exit price for {trade.symbol_name}")
            return

        #profit_loss = (exit_price - trade.entry_price) / trade.entry_price * 100
        profit_loss = (exit_price - trade.entry_price) 

        action_time = pd.Timestamp.now()
        logger.warning(f"Trade exited at {action_time} - {trade.symbol_name} at {exit_price}. {exit_type}. P/L: {profit_loss:.2f} points")

        # Mark the trade as exited and cancel the monitoring task
        trade.exited = True
        if trade.monitoring_task:
            trade_id = next((tid for tid, task in monitoring_tasks.items() if task == trade.monitoring_task), None)
            if trade_id:
                monitoring_tasks[trade_id].cancel()
                del monitoring_tasks[trade_id]
        
        trade_tracker.update_stats(profit_loss)

        # Place actual exit order using your brokerage API (replace with your actual order placement logic)
        # order_result = place_exit_order(trade.symbol_name, trade.position_type, lot_size, exit_price)  # Example
        # logger.info(f"Exit order placed: {order_result}")

    except Exception as e:
        logger.error(f"Error in exit_trade for {trade.token}: {e}")
        raise


async def execute_trade_logic(token, current_direction, previous_direction, second_direction): #########################################
    global current_positions, position_lock

    async with position_lock:
        # Check for active trades for the token
        active_trade = next((trade for trade in current_positions.values() if not trade.exited), None)

        # Check for exit signals first, even if there's an active trade
        if active_trade:
            exit_signal = (
                (active_trade.position_type == 'call_buy' and current_direction == -1 and previous_direction == 1) or
                (active_trade.position_type == 'put_buy' and current_direction == 1 and previous_direction == -1)
            )

            if exit_signal:
                try:
                    logger.warning("exiting")
                    await exit_trade(active_trade, "Regular Exit")
                except Exception as e:
                    print(f"Error exiting trade for {token}: {e}")

                # After exiting, set active_trade to None to allow for immediate new entry
                active_trade = None

       
        if not active_trade:
            entry_signal = False
            if current_direction == 1 and previous_direction == -1 and enable_call_trades and enable_all_trades and second_direction == current_direction: #########################################
                position_type = "call_buy"
                entry_signal = True
            elif current_direction == -1 and previous_direction == 1 and enable_put_trades and enable_all_trades and second_direction == current_direction: #########################################
                position_type = "put_buy"
                entry_signal = True

            if entry_signal:
                try:
                    logger.warning("entering normal trade.")
                    await enter_trade(token, position_type, "Regular Entry")
                except Exception as e:
                    print(f"Error entering trade for {token}: {e}")

2024-10-07 12:55:40,551 - ShoonyaApi - INFO - Starting candle_end_finder
2024-10-07 12:55:45,004 - ShoonyaApi - WARNING - resampled_data is empty.
2024-10-07 12:55:45,572 - ShoonyaApi - INFO - FeedJson for token 26009 is empty. LotSize: 15.
2024-10-07 12:55:50,582 - ShoonyaApi - INFO - FeedJson for token 26009 is empty. LotSize: 15.
2024-10-07 12:55:55,605 - ShoonyaApi - INFO - FeedJson for token 26009 is empty. LotSize: 15.
2024-10-07 12:56:00,002 - ShoonyaApi - WARNING - resampled_data is empty.
2024-10-07 12:56:00,631 - ShoonyaApi - INFO - FeedJson for token 26009 is empty. LotSize: 15.
2024-10-07 12:56:05,328 - ShoonyaApi - INFO - Feed Opened
2024-10-07 12:56:05,678 - ShoonyaApi - INFO - FeedJson for token 26009 is empty. LotSize: 15.
2024-10-07 12:56:05,683 - ShoonyaApi - INFO - WebSocket connected and subscribed successfully.
2024-10-07 12:56:15,007 - ShoonyaApi - INFO - Direction change detected for token 26009 at 2024-10-07 12:56:00: 0.0
2024-10-07 12:57:45,012 - ShoonyaApi - I

44108 not found in extra_feedJson


2024-10-07 13:07:30,005 - ShoonyaApi - INFO - Direction change detected for token 26009 at 2024-10-07 13:07:15: 1.0
2024-10-07 13:07:45,004 - ShoonyaApi - INFO - Direction change detected for token 26009 at 2024-10-07 13:07:30: -1.0
2024-10-07 13:07:45,006 - ShoonyaApi - WARNING - entering normal trade.
2024-10-07 13:07:45,012 - ShoonyaApi - ERROR - Unable to get entry price for BANKNIFTY09OCT24P50800


44108 not found in extra_feedJson


2024-10-07 13:11:00,023 - ShoonyaApi - INFO - Direction change detected for token 26009 at 2024-10-07 13:10:45: 1.0
2024-10-07 13:12:30,003 - ShoonyaApi - INFO - Direction change detected for token 26009 at 2024-10-07 13:12:15: -1.0
2024-10-07 13:12:30,005 - ShoonyaApi - WARNING - entering normal trade.
2024-10-07 13:12:30,008 - ShoonyaApi - ERROR - Unable to get entry price for BANKNIFTY09OCT24P50700


44105 not found in extra_feedJson


2024-10-07 13:12:45,003 - ShoonyaApi - INFO - Direction change detected for token 26009 at 2024-10-07 13:12:30: 1.0


In [ ]:
resampled_data

In [ ]:
trade1 = current_positions.get('26037')  # Fetch the trade object

if trade1:
    trade1.update_sl(new_sl_price=trade1.entry_price - 2)  # Update SL dynamically

In [ ]:
trade1 = current_positions.get('26009')  # Fetch the trade object
if trade1:
    
    trade1.update_trailing_sl(new_trailing_sl_price=trade1.highest_price - 1)  # Update Trailing SL

In [ ]:
trade1 = current_positions.get('26009')  # Fetch the trade object
if trade1:
   
    await trade1.force_exit()  # Force exit the trade if needed

In [ ]:
trade_tracker.print_summary()

In [ ]:
async def get_current_positions_info():
    async with position_lock:  # Acquire the lock to ensure data consistency
        for token, trade in current_positions.items():
            print(f"Token: {token}")
            print(f"  Position Type: {trade.position_type}")
            print(f"  Entry Price: {trade.entry_price}")
            print(f"  Stop Loss Price: {trade.sl_price}")
            print(f"  Trailing Stop Loss Price: {trade.trailing_sl_price}")
            print(f"  Trailing SL Activated: {trade.trail_activated}")
            print(f"  Trade Exited: {trade.exited}")  # Add this line
            print("------------------")
await get_current_positions_info()

In [ ]:
def option_subscription():
    op_chain = api.get_option_chain(exchange='NFO', tradingsymbol=state.ce_trading_symbol, strikeprice=atm_strike, count = 15)
    tokens = [item['token'] for item in op_chain['values']]
    subscriptions = [f"NFO|{token}" for token in tokens]
    # Subscribe to each token
    for sub in subscriptions:
        api.subscribe(sub)

    # Print the subscriptions for verification
    print("Subscribed to:", subscriptions)

option_subscription()

In [ ]:

def append_to_csv(trade_log):
    with open('trading_log.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow()

import pandas as pd

# Assuming resampled_data has the format {'token_name': DataFrame}
token, df = next(iter(completed_candles_dfs.items()))  # Get the token name and DataFrame

df.to_csv(f"{token}.csv", index=True)   # Save the DataFrame to a CSV file (include the index)
print(f"Saved data to {token}.csv")   